In [3]:
%pip install langchain-core langgraph>0.2.27
%pip install -qU langchain-google-vertexai
%pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatVertexAI(model = "gemini-1.5-pro", temperature = 0)

system_message = """
Eres la primera capa de un chatbot que ayuda a farmacéuticos a identificar productos que solicitan clientes que vienen a la farmacia. 
Tu objetivo es solamente identificar la categoría del producto que se está demandando.
Ejemplos:
- Input: "Jarabe para la tos seca apto para diabéticos."
  Respuesta esperada: "Jarabe"
- Input: "Una pasta de dientes para niños alérgicos al flúor"
  Respuesta esperada: "Pasta de dientes"
- Input: "Una crema hidratante con veneno de serpiente"
  Respuesta esperada: "Crema"
- Input: "Piruletas para la gargante"
  Respuesta esperada: "Piruletas"
"""

system_message = """
Eres la primera capa de un chatbot que ayuda a farmacéuticos a identificar productos que solicitan clientes que vienen a la farmacia. 
Tu objetivo es solamente identificar la categoría del producto que se está demandando. Debes elegir entre las siguientes categorias:
- Ortopedia: Productos de ortopedia como bastones, muletas, conteras, empuñadoras, fajas lumbares y fajas en general, collarines y cosas del estilo que se acoplen a partes del cuerpo humano o a utensilios que estos puedan utilizar.
- Ojos: Productos para ojos como colirio, gafas presbicia, gafas solares, soluciones lentes, limpieza ocular, parches oculares, otros óptica, audiología.

Ejemplos:
- Input: Andador regulable para persona mayor con problemas de cadera
  Respuesta esperada: Ortopedia
- Input: Collarin para lesión en el cuello
  Respuesta esperada: Ortopedia
- Input


- Input: "Jarabe para la tos seca apto para diabéticos."
  Respuesta esperada: "Jarabe"
- Input: "Una pasta de dientes para niños alérgicos al flúor"
  Respuesta esperada: "Pasta de dientes"
- Input: "Una crema hidratante con veneno de serpiente"
  Respuesta esperada: "Crema"
- Input: "Piruletas para la gargante"
  Respuesta esperada: "Piruletas"
"""

prompt = ChatPromptTemplate.from_messages(    
    [
    SystemMessage(content=system_message),
    MessagesPlaceholder(variable_name="messages") # Para mantener historial de conversación usamos MessagesPlaceholder
    ]
)

messages_static = [
    SystemMessage(content=system_message),
    HumanMessage(content="champú anticaida para cabellos grasos")
    ]

llm.invoke(messages_static)


AIMessage(content='Champú \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 158, 'candidates_token_count': 4, 'total_token_count': 162, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0006307683070190251, 'logprobs_result': {'top_candidates': [], 'chosen_candidates': []}}, id='run-54c32b32-eeb9-4b73-b4ba-4dfc07365662-0', usage_metadata={'inpu

In [34]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": response}

workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer = memory)

In [40]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

config = {"configurable": {"thread_id": "abc3456"}}

query = "Quiero una crema con veneno de serpiente"
input = [HumanMessage(content=query)]

output = app.invoke({"messages": input}, config)
print(output)

query2 = "Quiero una pasta de dientes para niños"
input2 = [HumanMessage(content=query2)]

output2 = app.invoke({"messages":input2},config)
print(output2)


{'messages': [HumanMessage(content='Quiero una crema con veneno de serpiente', additional_kwargs={}, response_metadata={}, id='9547852d-9565-4d71-afeb-bef1b730b407'), AIMessage(content='Crema \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'LOW', 'blocked': False, 'severity': 'HARM_SEVERITY_HIGH'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 156, 'candidates_token_count': 3, 'total_token_count': 159, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0027990